 # Pi parallel calculation using Montercarlo Method

#### ipyparallel tutorial http://people.duke.edu/~ccc14/sta-663-2016/19C_IPyParallel.html

In [ ]:
from ipyparallel import Client
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

### Main code

* __njobs__: number of times pi is going to be calculated in parallel using all engines.

In [ ]:
njobs = 1000

* __mc_pi__ function: accepts the max number of points created in the random distribution used to cacluate pi. It returns the value of pi calculated.   

In [ ]:
def mc_pi(n):
    x = np.random.uniform(-1, 1, (n,2))
    return 4*np.sum((x**2).sum(1) < 1)/n

### Connect to MPI cluster - Engine access 

In [ ]:
rc = Client(profile='mpi',timeout=30)

In [ ]:
rc.ids

* There are two main interfaces for accessing the engines:
> * With the direct interface, we access engines directly and explicitly with their identifiers.<br>__dv = rc[:]__<br>
> * With the load-balanced interface, we access engines through an interface that automatically and dynamically assigns work to appropriate engines.<br>__lv = rc.load_balanced_view()__

In [ ]:
dv = rc[:]

### Run parallel code 
#### With %%px magic command

* To run commands in parallel over all connected engines the __%px__ line magic or the  __%%px__ cell magic can be used. By default it runs code in all connected  engines but it can be customized to run code only in a subgroup of engines with __%%px -t 1,2__
* By default, the __%px__ magic executes commands in blocking mode; the cell only returns when the commands have completed on all engines. It is possible to run non-blocking commands with the -a option. In this case, the cell returns immediately, and the task's status and results can be polled asynchronously from IPython's interactive session

In [ ]:
%%px
import socket
import numpy as np
print (socket.gethostname())

#### With map_sync() method

* Runs in parallel #njobs times __mc_pi__ function in parallel using all engines. Wait synchronously to all engines to finish and collect results.<br> Use the __%%time__ magic to meassure time used in the calculation    

In [ ]:
%%time
res = dv.map_sync(mc_pi, [int(1e6)] * njobs)
print ("res array len: %s" %len(res))
print (res[:10])

### Result
#### Histogram

In [ ]:
plt.hist(res, 50,
         density=False,
         histtype='bar',
         facecolor='b',
         alpha=0.5)

plt.xlabel('Calculated pi')
plt.ylabel('Number of occurences')
plt.title('Openshift Python MPI - Pi calculation - Montercalo Method')
plt.show()

#### Statistical analysis
#### http://benalexkeen.com/basic-statistics-in-python/

In [ ]:
print ("Mean: %s" %np.mean(res))
print ("std:  %s" %np.std(res))
print ("ste:  %s" %stats.sem(res))

#### Deviation from exact value

In [ ]:
pi=np.mean(res)
error = abs(pi - np.pi)
print("Calculated pi is %.20f, error is %.20f" % (pi, error))